In [1]:
import numpy as np
import math
import scipy.optimize
import time

np.random.seed(5)
np.set_printoptions(precision=4)

In [2]:
def generateModel(locN, stageN):
    # Generate the prior distribution vector
    priorVec = np.random.random(locN)
    priorVec /= priorVec.sum()
    print('The prior probability vector is:\n' + str(priorVec) + '\n')

    # Generate the Markov transition probability matrix (left stochastic matrix; each column sums up to one)
    tranProbMat = np.random.rand(locN, locN)
    tranProbMat /= tranProbMat.sum(axis=0)[None,:]
    print('The Markov transition probability matrix (x=Px) is:\n' + str(tranProbMat) + '\n')

    # Generate the probability vector of successful detections (between 0.5 to 1.0)
    detectVec = 1 - 0.5 * np.random.random(locN)
    print('The probability vector of successful detections is:\n' + str(detectVec) + '\n')

    # Compute the probability vector of missed detections
    missVec = 1 - detectVec
    print('The probability vector of failed detections is:\n' + str(missVec) + '\n')
    
    # Return model
    return priorVec, tranProbMat, detectVec, missVec

In [3]:
# agentN - number of agents
agentN = 20
# locN - number of locations
locN = 500
# stageN - number of stages
stageN = 500

#q = [ 0.0255, 0.2603, 0.1463, 0.2415, 0.3264 ]
#missVec = [ 0.1024, 0.2454, 0.1862, 0.2387, 0.1829 ]

[priorVec, tranProbMat, detectVec, missVec] = generateModel(locN, stageN)
q = priorVec

The prior probability vector is:
[  8.8772e-04   3.4819e-03   8.2664e-04   3.6734e-03   1.9531e-03
   2.4463e-03   3.0627e-03   2.0731e-03   1.1869e-03   7.5067e-04
   3.2287e-04   2.9529e-03   1.7647e-03   6.3306e-04   3.5187e-03
   1.0960e-03   1.6565e-03   1.1840e-03   2.5144e-03   2.3187e-03
   2.3990e-03   1.0630e-03   1.1384e-03   1.0141e-03   1.3099e-03
   5.7649e-04   6.6226e-04   3.8546e-03   3.8398e-03   7.5344e-04
   9.7198e-05   8.1798e-04   2.7986e-03   3.1172e-03   9.1706e-05
   2.3100e-03   6.5668e-06   2.0613e-03   2.5584e-03   3.9414e-03
   1.0361e-03   3.2091e-03   3.4809e-03   3.6899e-03   8.8543e-06
   1.8774e-03   3.9247e-03   1.5953e-03   3.2540e-03   2.1852e-03
   3.0825e-03   1.9392e-03   1.1641e-04   3.4600e-04   4.4569e-04
   1.0047e-03   3.8585e-03   2.5263e-03   3.2657e-03   2.2637e-03
   2.5407e-03   3.2467e-03   3.7057e-03   3.6494e-03   3.2983e-03
   3.7670e-04   1.4438e-03   1.4199e-04   2.1848e-03   3.1836e-03
   2.0451e-04   7.5445e-04   1.4615e-03   9

In [4]:
start_time = time.time()
a = [ 0.0 ] * locN
lamb = 0
for k in range(locN):
    lamb = max(lamb, -q[k] * math.log(missVec[k]))
left = 0.0
right = lamb
itr = 1
while True:
    # Solve for a_k with current lamb
    for k in range(locN):
        if (q[k] * math.log(missVec[k]) + lamb >= 0):
            a[k] = 0;
        else:
            tmp = -lamb / (q[k] * math.log(missVec[k]))
            a[k] = math.log(tmp, missVec[k])
    if math.fabs(sum(a) - agentN) <= 1e-18:
        print('Found optimal solution at iteration ' + str(itr))
        break
    elif sum(a) > agentN: # Increase lamb
        tmp = lamb
        lamb = (lamb + right) / 2.0
        left = tmp
    else: # Decrease lamb
        tmp = lamb
        lamb = (left + lamb) / 2.0
        right = tmp
    #print('Current gap = ' + str(sum(a) - agentN) + \
    #      ', lamb = ' + str(lamb) + ', left = ' + str(left) + ', right = ' + str(right))
    #print(right-left)
    itr += 1
objVal = sum([q[k] * math.pow(missVec[k], a[k]) for k in range(locN)])
print('Optimal value: ' + str(objVal))
print('Time taken to run: ' + str(time.time() - start_time))
print('Optimal decision variables are: ' + str(a))

Found optimal solution at iteration 54
Optimal value: 0.874090519168
Time taken to run: 0.06864094734191895
Optimal decision variables are: [0, 0.021495478796664694, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.005397242460124239, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.12189245544081936, 0, 0, 0.09768693552091239, 0.27902163434199917, 0.15815638154955008, 0, 0, 0, 0, 0.03468371438351174, 0.05117702094504648, 0, 0.0038536210554055666, 0, 0, 0, 0, 0.05411705564700448, 0, 0.2543755079469486, 0, 0, 0.03543128284201918, 0, 0.22066437675296183, 0.25745743761128137, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.18046183138072378, 0, 0, 0, 0, 0, 0.3113359659059356, 0, 0.15718662579851994, 0, 0, 0, 0, 0.25631476444904766, 0, 0, 0, 0, 0, 0.12065536486083864, 0, 0, 0, 0, 0, 0, 0.3249977410131434, 0, 0, 0, 0, 0, 0, 0, 0.13231611324327627, 0.20403513336170157, 0, 0, 0, 0, 0, 0, 0, 0.31983262103067955, 0, 0.24724519702578354, 0.06199810032373494, 0, 0.08403845476135961, 0.30238700297667914, 